# [Auto-Encoding Variational Bayes](https://arxiv.org/abs/1312.6114)のメモ

## モチベーション

intractableな事後分布と潜在的な変数をもつグラフィカルモデルを効率的に推論・学習したい。

このpaperで述べられていることは2つ:

1. 変分下界のreparameterizationを示す、これによって確率的勾配降下法を用いて直感的に最適化を行うことのできる変分下界のestimator(Stochastic Gradient Variational Bayes(SGVB)が得られる
2. 提案する変分下界のestimatorを用いて、各データ点について潜在的な変数を持つi.i.d.なデータセットに対し、近似推論モデル(recognition model)をintractableな事後分布にフィッティングすることで効率的に事後分布の推論が行えるアルゴリズム-Auto Encoding VB(AEVB)を示す

### 問題

従来の変分ベイズ法はintractableな事後分布に対しこれを近似した分布を用いて推測を行う。この事後分布の平均場近似は、近似した事後分布についての期待値が解析的に求めることができることが条件となるが、近似した事後分布自体がintractableなことが実際には多い。

提案するSGVBは変分下界のreparameterizationによって微分可能でunbiasedなestimatorを与える、このestimatorを用いれば殆どどんなモデルに対しても事後分布の効率的な近似を行うことができ、また確率的勾配降下法を用いてこのモデルに対し直感的な最適化を行うことができる。

## 手法

### 前提

* 各データ点について潜在的な変数を持つi.i.d.なデータセット
* グローバルパラメータについて、ML/MAP推定したい
* 潜在変数について変分推定(?)したい

Appendixにはグローバルパラメータの変分推定についても記載あり

### 問題設定

* 確率変数$x$からの$N$個のサンプルから成るi.i.d.なデータセット$X = \{x^{(i)}\}_{i=1}^N$を考える
* このデータは観測できない確率変数$z$が介在する確率過程から生成されたと想定する
* 周辺尤度も事後分布もその平均場近似も皆intractableな場合を考える
* データセットのサイズが大きく、Monte Carlo EMなどのサンプリングに基づく推測が現実的でない場合を考える

アプローチとしては、intractableな事後分布$p_{\theta}(z|x)$を近似する推論(recognition)モデル$q_{\theta}(z|x)$を導入して、推論モデルのパラメーター$\phi$と生成モデルのパラメーター$\theta$を同時に学習する。

### 変分下界

(ここはこのpaperに限らず一般的な話)

周辺尤度$\log p_{\theta}(x^{(1)}, \cdots, x^{(N)}) = \sum_{i=1}^N \log p_{\theta}(x^{(i)})$の各々は以下で表すことができる。

$$
\log p_{\theta}(x^{(i)}) = \mathcal{L}(\theta, \phi; x^{(i)}) + D_{KL}(q_{\phi}(z|x^{(i)}) || p_{\theta}(z|x^{(i)}))
$$

ただし

$$
\mathcal{L}(\theta, \phi; x^{(i)}) = \int q_{\phi}(z|x^{(i)}) \log \frac{p_{\theta}(x^{(i)}, z)}{q_{\phi}(z|x^{(i)})} dz
$$

---

式の展開について:

まず

$$
\begin{eqnarray}
\log p_{\theta}(x^{(i)}) &=& \log \int p_{\theta}(x^{(i)}, z)dz \\
&=& \log \int q_{\phi}(z|x^{(i)}) \frac{p_{\theta}(x^{(i)}, z)}{q_{\phi}(z|x^{(i)})} dz \\
&\geq& \int q_{\phi}(z|x^{(i)}) \log(\frac{p_{\theta}(x^{(i)}, z)}{q_{\phi}(z|x^{(i)})}) dz \\
&=& \mathcal{L}(\theta, \phi; x^{(i)})
\end{eqnarray}
$$

不等式の部分はイエンセンの不等式を用いた。この式の両辺の差を調べると

$$
\begin{eqnarray}
\log p_{\theta}(x^{(i)}) - \mathcal{L}(\theta, \phi; x^{(i)}) &=& \int q_{\phi}(z|x^{(i)})dz \log p_{\theta}(x^{(i)}) - \int q_{\phi}(z|x^{(i)}) \log(\frac{p_{\theta}(x^{(i)}, z)}{q_{\phi}(z|x^{(i)})}) dz \\
&=& \int q_{\phi}(z|x^{(i)}) \log(\frac{p_{\theta}(x^{(i)}) q_{\phi}(z|x^{(i)})}{p_{\theta}(x^{(i)}, z)}) dz \\
&=& \int q_{\phi}(z|x^{(i)}) \log(\frac{p_{\theta}(x^{(i)}) q_{\phi}(z|x^{(i)})}{p_{\theta}(z|x^{(i)})p_{\theta}(x^{(i)})}) dz \\
&=& \int q_{\phi}(z|x^{(i)}) \log(\frac{q_{\phi}(z|x^{(i)})}{p_{\theta}(z|x^{(i)})}) dz \\
&=& D_{KL}(q_{\phi}(z|x^{(i)}) || p_{\theta}(z|x^{(i)}))
\end{eqnarray}
$$

---

変分下界$\mathcal{L}(\theta, \phi; x^{(i)})$について、

$$
\begin{eqnarray}
\mathcal{L}(\theta, \phi; x^{(i)}) &=& \int q_{\phi}(z|x^{(i)}) \log \frac{p_{\theta}(x^{(i)}, z)}{q_{\phi}(z|x^{(i)})} dz \\
&=& \int q_{\phi}(z|x^{(i)}) \log \frac{p_{\theta}(x^{(i)}| z) p_{\theta}(z)}{q_{\phi}(z|x^{(i)})} dz \\
&=& - \int q_{\phi}(z|x^{(i)}) \log \frac{q_{\phi}(z|x^{(i)})}{p_{\theta}(z)}dz + \int q_{\phi}(z|x^{(i)}) \log p_{\theta}(x^{(i)}|z)dz \\
&=& - D_{KL}(q_{\phi}(z|x^{(i)}) || p_{\theta}(z)) + \mathbb{E}_{q_{\theta}(z|x^{(i)})} [ \log p_{\theta}(x^{(i)}|z)]
\end{eqnarray}
$$

### SGVB

特定の条件下では近似事後分布$q_{\theta}(z|x)$の確率変数$\widetilde{z} \sim q_{\phi}(z|x)$を微分可能な変換$g_{\phi}( \epsilon, x)$でreparamiterizeすることができる($\epsilon$はノイズ)。

$$
\widetilde{z} = g_{\phi}( \epsilon, x), \epsilon \sim p(\epsilon)
$$

これを用いれば、ある関数$f(z)$の$q_{\phi}(z|x)$に関する期待値をモンテカルロ法に基いて見積もることができる:

$$
\mathbb{E}_{q_{\theta}(z|x^{(i)})} [f(z)] = \mathbb{E}_{p(\epsilon)}[f(g_{\phi}(\epsilon, x^{(i)}))] \simeq \frac{1}{L} \sum_{i=1}^L f(g_{\phi}(\epsilon^{(l)}, x^{(i)})), \epsilon^{(l)} \sim p(\epsilon)
$$

この手法を変分下界に適用すればSGVBが得られる:

$$
\widetilde{\mathcal{L}}^B(\theta, \phi; x^{(i)}) = -D_{KL}(q_{\phi}(z|x^{(i)})||p_{\theta}(z)) + \frac{1}{L} \sum_{l=1}^L (\log p_{\theta}(x^{(i)}|z^{(i, l)})
$$

この式は、$D_{KL}(q_{\phi}(z|x^{(i)})||p_{\theta}(z))$が解析的に求めることが出来て、復元誤差$\mathbb{E}_{q_{\theta}(z|x^{(i)})} [ \log p_{\theta}(x^{(i)}|z)]$のみがサンプリングを必要とする場合の式。より一般的な場合の式はpaperの式(6)を用いる。

この式ではKL情報量は$\phi$の正則化項と捉えることができる、この項は近似事後分布が真の事前分布$p_{\theta}(z)$により近づくよう仕向けている。